In [1]:
from azureml.core import Workspace, Dataset

subscription_id = '18522758-626e-4d88-92ac-dc9c7a5c26d4'
resource_group = 'Analytics.Aml.Experiments.Workspaces'
workspace_name = 'aa-ml-aml-workspace'

ws = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(ws, name='hr-employee-attrition')
attritionData = dataset.to_pandas_dataframe()

In [2]:
attritionData.head(5)

,Column1,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,41,True,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,...,1,80,0,8,0,1,6,4,0,5
1,1,49,False,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,...,4,80,1,10,3,3,10,7,1,7
2,2,37,True,Travel_Rarely,1373,Research & Development,2,2,Other,1,...,2,80,0,7,3,3,0,0,0,0
3,3,33,False,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,...,3,80,0,8,3,3,8,7,3,0
4,4,27,False,Travel_Rarely,591,Research & Development,2,1,Medical,1,...,4,80,1,6,3,3,2,2,2,2


In [3]:
from azureml.core import Experiment

cluster_name = "aml-trainer-gpu"
compute_target = ws.compute_targets[cluster_name]

experiment_name = "employee-attrition-model-Py"
exp = Experiment(workspace=ws, name=experiment_name)

In [5]:
from azureml.train.estimator import Estimator

est = Estimator(source_directory=".", 
                entry_script='train.py', 
                # pass dataset object as an input with name 'hr_employee_attrition'
                inputs=[dataset.as_named_input('hr_employee_attrition')],
                compute_target=compute_target,
                pip_packages=["scikit-learn==0.20.3","sklearn-pandas","joblib","interpret","azureml-dataprep[fuse,pandas]",
                             "pandas", "matplotlib", "seaborn"])

In [6]:
from azureml.widgets import RunDetails

run = exp.submit(est)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…